## 1. Install the necessary libraries:

In [68]:
#!pip install pyaudio pytube SpeechRecognition nltk pydub ffmpeg-python

## 2. Import the required libraries in your Python script:

In [69]:
import pytube
from pydub import AudioSegment
from pydub.silence import split_on_silence 
import speech_recognition as sr
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import nltk
import time
import os

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 3. Create a function to retrieve the YouTube video's audio:

In [70]:
def get_audio(video_url,output_path='.',src='audio.mp4'):
    youtube = pytube.YouTube(video_url,use_oauth=True,allow_oauth_cache=True )
    audio = youtube.streams.get_audio_only()
    audio.download(output_path=output_path, filename=src)

In [71]:
#get_audio('https://www.youtube.com/watch?v=yWMKYID5fr8')

## 4. Convert_audio_format 

In [72]:
def format_audio(src='audio.mp4',dst='audio.wav', Input_format='mp4', output_format="wav"):
    audio = AudioSegment.from_file(src,format=Input_format)
    audio.export(dst, format=output_format)

In [73]:
get_audio('https://www.youtube.com/watch?v=wZvVNGxoGq4')
format_audio()

## 5. Create a function to convert the audio to text:

In [74]:
"""
r = sr.Recognizer()
source = 'audio.wav'
audio_text = ''
#audio = r.record(source, 
                 #duration=4
#                 )
audio = AudioSegment.from_wav(source) 
chunks = split_on_silence(audio, 
                          # must be silent for at least 0.5 seconds 
                          # or 500 ms. adjust this value based on user 
                          # requirement. if the speaker stays silent for 
                          # longer, increase this value. else, decrease it. 
                          min_silence_len = 5000, 
                          # consider it silent if quieter than -16 dBFS 
                          # adjust this per requirement 
                          silence_thresh = -16
                          )
i = 0
for chunk in chunks:
    i += 1
    wait_time = 0
    time.sleep(wait_time)
    
    # specify the bitrate to be 192 k 
    chunk.export("./audio_chunk.wav", bitrate ='192k', format ="wav") 
    #chunk.export(f"./audio_chunk{i}.wav", bitrate ='192k', format ="wav")  
    
    # the name of the newly created chunk 
    filename = 'audio_chunk'+'.wav' 
    with sr.AudioFile(filename) as source:
         #r.adjust_for_ambient_noise(source) 
         #audio_listened = r.listen(source) 
         audio_listened = r.record(source)
    try: 
        actual_text = r.recognize_google(audio_listened, language='en-US')
        audio_text += actual_text
    except:
       audio_text += ' Error ' 
"""

'\nr = sr.Recognizer()\nsource = \'audio.wav\'\naudio_text = \'\'\n#audio = r.record(source, \n                 #duration=4\n#                 )\naudio = AudioSegment.from_wav(source) \nchunks = split_on_silence(audio, \n                          # must be silent for at least 0.5 seconds \n                          # or 500 ms. adjust this value based on user \n                          # requirement. if the speaker stays silent for \n                          # longer, increase this value. else, decrease it. \n                          min_silence_len = 5000, \n                          # consider it silent if quieter than -16 dBFS \n                          # adjust this per requirement \n                          silence_thresh = -16\n                          )\ni = 0\nfor chunk in chunks:\n    i += 1\n    wait_time = 0\n    time.sleep(wait_time)\n    \n    # specify the bitrate to be 192 k \n    chunk.export("./audio_chunk.wav", bitrate =\'192k\', format ="wav") \n    #chunk.expo

In [75]:
def convert_audio_to_text(source='audio.wav',chunk_duration=5000):

    r = sr.Recognizer()
    audio_text = ''
    #audio = r.record(source, 
                 #duration=4
    #                 )
    audio = AudioSegment.from_wav(source) 
    chunk_duration = chunk_duration

    start = 0
    end = chunk_duration
    i = 0
    for _ in range( int( len(audio)/chunk_duration ) ):
        chunk = audio[start:end]
        chunk.export("./audio_chunk.wav", format ="wav") 
        #chunk.export("./audio_chunk.wav", bitrate ='192k', format ="wav") 
        start += chunk_duration
        end += chunk_duration
        i += 1

        #chunk.export(f"./audio_chunk{i}.wav", bitrate ='192k', format ="wav")  
    
        # the name of the newly created chunk 
        temporal_filename = './audio_chunk'+'.wav' 
        with sr.AudioFile(temporal_filename) as source:
             #r.adjust_for_ambient_noise(source) 
             #audio_listened = r.listen(source) 
             audio_listened = r.record(source)

        try: 
            actual_text = r.recognize_google(audio_listened, language='en-US')
            audio_text += actual_text + ' '
        except:
           audio_text += 'Error' 

    os.remove(temporal_filename)

    return audio_text

## 6. Create a function to generate a summary of the text:

In [76]:
def generate_summary(text,n_top=5):
    sentences = sent_tokenize(text)
    stopwords_list = set(stopwords.words("english"))
    words = word_tokenize(text)
    words_cleaned = [word.lower() for word in words if word.isalnum() and word.lower() not in stopwords_list]
    fdist = FreqDist(words_cleaned)
    summary = fdist.most_common(n_top)  # Generate a summary of the top 5 most frequent words
    return summary

## 7. Create the graphical interface using Tkinter:

In [77]:
import tkinter as tk

def generate_summary_button_clicked():
    try:
        ouput_label.config(text='Procesando')
        video_url = video_url_entry.get()
        get_audio(video_url)
        format_audio()
        audio_text = convert_audio_to_text()
        summary = generate_summary(audio_text)
        # Display the summary in a text box or any other appropriate way
        ouput_label.config(text=summary)
        # Schedule the next update after 1 second (1000 milliseconds)
        #window.after(1000, update_label)
        #print(summary)
    except Exception as e:
        #print(e.message, e.args)
        ouput_label.config(text=e)

# Create the main window
window = tk.Tk()
window.title("YouTube Video Keywords Generator")

# Create the video URL label and entry field
video_url_label = tk.Label(window, text="Video URL:")
video_url_label.pack()
video_url_entry = tk.Entry(window)
video_url_entry.pack()
# Create a label with the text to display
ouput_label = tk.Label(window, text="Ouput")
ouput_label.pack()

# Create the Generate Summary button
generate_summary_button = tk.Button(window, text="Get keywords", command=generate_summary_button_clicked)
generate_summary_button.pack()

# Start the main loop
window.mainloop()